## Семинар 1 Индекс

## Intro

### чтение файла 
- конструкция __with open__ (recommended)
- конструкция __open + close__

In [211]:
fpath = 'fpath.txt'

# одним массивом  
with open(fpath, 'r') as f:  
    text = f.read() 

#по строкам, в конце каждой строки \n  
with open(fpath, 'r') as f:   
    text = f.readlines() 

#по строкам, без \n   
with open(fpath, 'r') as f:   
    text = f.read().splitlines() 
    
#not reccomended  
file = open(txt_fpath, 'r')  
text = file.read()    
file.close() 

### работа с файлами и папками

### os.path  
путь до файла

In [ ]:
import os

# возвращает полный путь до папки/файла по имени файла / папки
print(os.path.abspath('fpath.txt'))

# возвращает имя файла / папки по полному пути до него
print(os.path.basename('/your/path/to/folder/with/fpath.txt'))

# проверить существование директории - True / False
print(os.path.exists('your/path/to/any/folder/'))

### os.listdir  
возвращает список файлов в данной директории

In [ ]:
main_dir = '/your/path/to/folder/with/folders/'
os.listdir(main_dir)

сделаем пути абсолютными, чтобы наш код не зависел от того, где лежит этот файл

In [ ]:
[main_dir + fpath for fpath in os.listdir(main_dir)]

не забывайте исключать системные директории, такие как .DS_Store

In [ ]:
[main_dir + fpath for fpath in os.listdir(main_dir) if not '.DS_Store' in fpath]

### os.walk
root - начальная директория  
dirs - список поддиректорий (папок)   
files - список файлов в этих поддиректориях  

In [ ]:
main_dir = '/your/path/to/folder/with/folders/'

for root, dirs, files in os.walk(main_dir):
    for name in files:
        print(os.path.join(root, name))

> __os.walk__ возвращает генератор, это значит, что получить его элементы можно только проитерировавшись по нему  
но его легко можно превратить в list и увидеть все его значения

In [ ]:
list(os.walk(main_dir))

##  Обратный индекс 

Сам по себе обратный индекс не может осуществлять поиск, для этого необходимо добавить к нему определенную метрику. Это не совсем очевидная задача, поэтому немного отложим ее. А сейчас посмотрим, что полезного можно вытащить из индекса.    
По сути, индекс - это информация о частоте встречаемости слова в каждом документе.   
Из этого можно понять, например:
1. какое слово является самым часто употребимым / редким
2. какие слова встречаются всегда вместе. Так можно парсить твиттер, fb, форумы и отлавливать новые устойчивые выражения в речи
3. какой документ является самым большим / маленьким (очень изощренный способ, когда есть _len_)

### __Задача__: 
получите обратный индекс для коллекция документов.    
Перед этим постройте матрицу терм-документ и сделайте функцию булева поиска, которая по запросу будет возвращать 5 релевантных документов.   
В качестве коллекции возьмите сценарий сезонов сериала Друзья. Одна серия - один документ.

[download_friends_corpus](https://yadi.sk/d/k_M7n63A3adGSz)

Этапы:   
    1. получить коллекцию документов
    2. для каждого файла коллекции сделать необходимую на ваш взгляд предобработку
    3. получить матрицу терм-документ, написать функцию поиска по ней
    4. получить обратный индекс в виде словаря, где ключ - нормализованное слово, 
    значение - список файлов, в которых это слово встречается
    5. вывести кусочек индекса в виде таблицы 
    6. сделать анализ обратного индекса. Это задание принимается в виде кода и ответов на вопросы

![](Friends/wedding.png)

Напоминание:    
> При итерации по списку вы можете помимо самого элемента получить его порядковый номер    
``` for i, element in enumerate(your_list): ...  ```    
Иногда для получения элемента делают так -  ``` your_list[i] ```, старайтесь этого избегать

In [2]:
import os

In [21]:
main_dir = 'D:\\MyDocuments\\proga\\Инфопоиск\\Friends\\'
files_list = []
print(os.path.exists(main_dir))

### пройдитесь по всем папкам коллекции и соберите все пути .txt файлов
i = 0
for root, dirs, files in os.walk(main_dir):
    for name in files:
        files_list.append(os.path.join(root, name))
        i += 1

print(files_list[0:2])
        
### _check : в коллекции должно быть 165 файлов
print(i)

True
['D:\\MyDocuments\\proga\\Инфопоиск\\Friends\\Friends - season 1\\Friends - 1x01 - The One Where Monica Gets A Roommate.ru.txt', 'D:\\MyDocuments\\proga\\Инфопоиск\\Friends\\Friends - season 1\\Friends - 1x02 - The One With The Sonogram At The End.ru.txt']
165


### Некоторая предобработка текстов

In [22]:
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()


def normalize(text):
    """
    функция нормализации
    
    ::парметры::
    @text - ненормализованный текст (string)
    
    ::returns::
    нормализованный текст (string)
    """
    
    tokens = [word for word in word_tokenize(text) if word.isalnum()]
    lemmas = [morph.parse(token)[0].normal_form for token in tokens]
    
    return ' '.join(lemmas)

In [27]:
texts = {}
for fpath in files_list:
    with open(fpath, 'r', encoding='utf-8') as f:  
        text = f.read()
        texts[fpath] = text

In [41]:
i = 0
norm_texts = {}
for t in texts.values():
    norm_t = normalize(t)
    norm_texts[list(texts.keys())[i]] = norm_t
    i += 1

In [43]:
norm_texts['D:\\MyDocuments\\proga\\Инфопоиск\\Friends\\Friends - season 1\\Friends - 1x01 - The One Where Monica Gets A Roommate.ru.txt']

'как весь начаться да нечего рассказывать он просто сотрудник ладный ты ты же на свидание с он собраться значит он не мочь не быть с придурь джой вести себя прилично так у он горб и парик в придача погодить я знать он есть мел ну знаете я просто не хотеть чтобы она пережить то же что я с ой ладный вы успокоиться успокоиться это даже не свидание просто двое молодая человек идти поужинать вместе без секс у я весь такой ну вот я снова старшеклассник стоять посреди столовый и вдруг понимать что я полностью голый да да мы тоже это сниться значит смотреть я вниз и видеть что у я телефон там вместо вот именно такой я не сниться и вдруг внезапно телефон начинать звонить я не знать что делать весь смотреть на я а до тот не смотреть наконец я догадываться что хороший быть ответить и оказываться что это мой мать и и есть самый странный потому что она никогда я не звонить когда он здороваться я в ответ хотеться повеситься ты в порядок милый я чувствовать себя так вроде залезть к я в глотка схватит

### постройте матрицу терм-документ

In [46]:
#term_doc_matrix = []

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()
X = vec.fit_transform(list(norm_texts.values()))
df = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())
print(df)

     000  007  009  038  03815  10  100  1000  101  102 ...   ящичек  ёй  \
0      0    0    0    0      0   0    0     0    0    0 ...        0   0   
1      0    0    0    0      0   0    0     0    0    0 ...        0   0   
2      0    0    0    0      0   0    2     0    0    0 ...        0   0   
3      0    0    0    0      0   1    0     0    0    0 ...        0   0   
4      0    0    0    0      0   0    0     0    0    0 ...        0   0   
5      0    0    0    0      0   0    0     0    0    0 ...        0   0   
6      0    0    0    0      0   0    0     0    0    0 ...        0   0   
7      0    0    0    0      0   0    0     0    0    0 ...        0   0   
8      0    0    0    0      0   0    0     0    0    0 ...        0   0   
9      0    0    0    0      0   0    0     0    0    0 ...        0   0   
10     0    0    0    0      0   0    0     0    0    0 ...        0   0   
11     0    0    0    0      0   0    0     0    0    0 ...        0   0   
12     0    

In [50]:
term_doc_matrix = df.as_matrix()
term_doc_matrix
df.index.tolist()

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164]

Тут я сначала просто тренируюсь, как написать Булев поиск, ***этот код смотреть не надо***. Я его оставила, потому что хотела показать, что изначально я хотела работать со структурой дерева. То есть сделать из запроса дерево, а потом обходить его postorder traverse. Я нашла модуль luqum, который парсит в деревья запросы с логическими переменными, но я не смогла написать traverse для этого дерева. Во-первых, оно не бинарное, а во-вторых, у этого модуля почему-то нет функций, которыми можно просто вызывать соседей справа и слева или родителей и тд.

In [119]:
import re

In [120]:
class Node:
    def __init__(self,key):
        self.left = None
        self.right = None
        self.val = key
        
def printPostorder(root):
 
    if root:
 
        # First recur on left child
        printPostorder(root.left)
 
        # the recur on right child
        printPostorder(root.right)
 
        # now print the data of node
        print(root.val),

In [168]:
from luqum.parser import parser
from luqum.utils import LuceneTreeVisitor


query = '(Моника ИЛИ Фиби) & Рэйчел & (НЕ Джои) & Росс'


res = re.sub('ИЛИ', 'OR', query)
res = re.sub('&', 'AND', res)
res = re.sub('НЕ', 'NOT', res)
tree = parser.parse(res)
tree = repr(tree)
print(tree)
root_oper = re.search('(.*?)\(', tree).group(1)
layer1 = re.search('.*?\((.*)\)\)', tree).group(1).split(', ')
print(layer1)
for element in layer1:
    print(element)
    if element.startswith('Group'):
        layer2_root = re.search('Group\((.*?)\(', element).group(1)
        layer2 = re.search(layer2_root + '\((.*)\)', element).group(1)
        print(layer2)
    else:
        word = re.search('\'(.*)\'', element).group(1)
        print(word)
#root.left = Node('k')


AndOperation(Group(OrOperation(Word('Моника'), Word('Фиби'))), Word('Рэйчел'), Group(Not(Word('Джои'))), Word('Росс'))
["Group(OrOperation(Word('Моника')", "Word('Фиби')))", "Word('Рэйчел')", "Group(Not(Word('Джои')))", "Word('Росс'"]
Group(OrOperation(Word('Моника')
Word('Моника'
Word('Фиби')))
Фиби
Word('Рэйчел')
Рэйчел
Group(Not(Word('Джои')))
Word('Джои'))
Word('Росс'
Росс


In [173]:
# '(Моника ИЛИ Фиби) & Рэйчел & (НЕ Джои) & Росс'
query = '(НЕ Моника) & Фиби & Рэйчел & Чендлер & Джои & (НЕ Росс)' 
words_and = re.split(' & ', query)
print(words_and)
last_end = []
for element in words_and:
    m = re.search('\((.*)\)', element)
    if m:
        n = re.search('(ИЛИ|НЕ|&)', m.group(1))
        if n:
            if n.group(1) == 'ИЛИ':
                words_or = re.split(' .+? ', m.group(1))
                set_or1 = []
                set_or2 = []
                
                # print(df[normalize(words_or[0])] > 0)
                
                for i, element in enumerate(df[normalize(words_or[0])] > 0):
                    if element == True:
                        set_or1.append(i)
                
                for i, element in enumerate(df[normalize(words_or[1])] > 0):
                    if element == True:
                        set_or2.append(i)
                        
                set_or = set(set_or1).union(set(set_or2))
                print(set_or)
            elif n.group(1) == '&':
                words_and2 = re.split(' .+? ', m.group(1))
                set_and1 = []
                set_and2 = []
                
                for i, element in enumerate(df[normalize(words_and2[0])] > 0):
                    if element == True:
                        set_and1.append(i)
                
                for i, element in enumerate(df[normalize(words_and2[1])] > 0):
                    if element == True:
                        set_and2.append(i)
                        
                set_and = set(set_and1).intersection(set(set_and2))
                print(set_and)
            else:
                neg = re.search('НЕ (.*)', m.group(1))
                neg_word = neg.group(1)
                print(neg_word)
                set_neg = []
                for i, element in enumerate(df[normalize(words_and2[0])] > 0):
                    if element == False:
                        set_neg.append(i)
                set_neg = set(set_neg)
                print(set_neg)
    else:
        last_end.append(element)
        
print(last_end)
set_and1 = []
set_and2 = []
    
                
for i, element in enumerate(df[normalize(last_end[0])] > 0):
    if element == True:
        set_and1.append(i)
                
for i, element in enumerate(df[normalize(last_end[1])] > 0):
    if element == True:
        set_and2.append(i)
                        
set_and_set = set(set_and1).intersection(set(set_and2))
print(list(set_and_set)[0:5])

['(НЕ Моника)', 'Фиби', 'Рэйчел', 'Чендлер', 'Джои', '(НЕ Росс)']
Моника
{11, 14, 151, 25, 30, 32, 160, 48, 52, 56, 67, 74, 76, 81, 83, 84, 86, 101, 107, 108, 109, 112, 113, 125}
Росс
{11, 14, 151, 25, 30, 32, 160, 48, 52, 56, 67, 74, 76, 81, 83, 84, 86, 101, 107, 108, 109, 112, 113, 125}
['Фиби', 'Рэйчел', 'Чендлер', 'Джои']
[128, 1, 130, 132, 5]


### В итоге получилась вот такая функция поиска

In [174]:
### напишите функцию булева поиска по построенной матрице
def boolean_search(df, query) -> list:
    """
    Produces a Boolean search with respect to the term-document matrix
    :return: list of first 5 relevant documents
    """
    words_and = re.split(' & ', query)
    last_end = []
    for element in words_and:
        m = re.search('\((.*)\)', element)
        if m:
            n = re.search('(ИЛИ|НЕ|&)', m.group(1))
            if n:
                if n.group(1) == 'ИЛИ':
                    words_or = re.split(' .+? ', m.group(1))
                    set_or1 = []
                    set_or2 = []
                    for i, element in enumerate(df[normalize(words_or[0])] > 0):
                        if element == True:
                            set_or1.append(i)

                    for i, element in enumerate(df[normalize(words_or[1])] > 0):
                        if element == True:
                            set_or2.append(i)

                    set_or = set(set_or1).union(set(set_or2))
                elif n.group(1) == '&':
                    words_and2 = re.split(' .+? ', m.group(1))
                    set_and1 = []
                    set_and2 = []

                    for i, element in enumerate(df[normalize(words_and2[0])] > 0):
                        if element == True:
                            set_and1.append(i)

                    for i, element in enumerate(df[normalize(words_and2[1])] > 0):
                        if element == True:
                            set_and2.append(i)

                    set_and = set(set_and1).intersection(set(set_and2))
                else:
                    neg = re.search('НЕ (.*)', m.group(1))
                    neg_word = neg.group(1)
                    set_neg = []
                    for i, element in enumerate(df[normalize(words_and2[0])] > 0):
                        if element == False:
                            set_neg.append(i)
                    set_neg = set(set_neg)
        else:
            last_end.append(element)

    set_and1 = []
    set_and2 = []

    for i, element in enumerate(df[normalize(last_end[0])] > 0):
        if element == True:
            set_and1.append(i)

    for i, element in enumerate(df[normalize(last_end[1])] > 0):
        if element == True:
            set_and2.append(i)

    set_and_set = set(set_and1).intersection(set(set_and2))
    
    return list(set_and_set)[0:5]


#запросы 
input_text = [
    'Моника & Фиби & Рэйчел & Чендлер & Джои & Росс',
    '(Моника ИЛИ Фиби) & Рэйчел & (Чендлер ИЛИ Джои) & Росс', 
    '(НЕ Моника) & Фиби & Рэйчел & Чендлер & Джои & (НЕ Росс)'
]

<img src="pics/inv_index3.svg" alt="Drawing" style="width: 600px;"/>

Совет для построения обратного индекса: 
> В качестве словаря используйте ``` defaultdict ``` из модуля collections   
Так можно избежать конструкции ``` dict.setdefault(key, default=None) ```

In [192]:
from collections import defaultdict, Counter
def inverted_index(norm_texts) -> dict:
    """
    Create inverted index by input doc collection
    :return: inverted index
    """
    index = defaultdict(list)
    i = 0
   
    for text in norm_texts.values():
        for word in text.split(' '):
            if i not in index[word]:
                index[word].append(i)
        i += 1

    return index

print(inverted_index(norm_texts))

defaultdict(<class 'list'>, {'40': [7, 20, 98, 115, 147], 'волшебник': [52], 'отступиться': [113], 'шутить': [0, 2, 4, 7, 8, 12, 15, 16, 17, 18, 27, 28, 37, 40, 41, 43, 46, 52, 59, 72, 73, 76, 82, 85, 89, 92, 94, 95, 96, 97, 104, 105, 108, 110, 111, 112, 116, 119, 120, 122, 130, 133, 134, 135, 136, 137, 144, 146, 149, 152, 157, 158, 160, 161, 162, 163, 164], 'коляска': [11, 62, 151], 'night': [62, 89], 'чрезвычайно': [83], 'устраиваться': [2, 128], 'ночной': [7, 86, 110, 122, 126], 'измельчить': [57], 'бергман': [53], 'должница': [125], 'перебить': [4, 61, 75, 95, 119], 'чистить': [12, 21, 62, 67, 112, 154], 'кейт': [50, 137, 161], 'мэррилла': [15], 'чулок': [64, 164], 'аббатство': [119], 'соглашение': [147], 'эмиль': [4, 17, 21, 30, 36, 43, 66, 67, 75, 76, 80, 83, 90, 113, 118, 119, 122, 133, 139, 155], 'чёрный': [13, 14, 16, 20, 22, 30, 32, 38, 54, 103, 107, 109, 110, 136, 138, 140, 162], 'отстойный': [47], 'кантри': [92], 'пополам': [0, 4, 74, 88, 93, 117, 119, 156, 163], 'слава': [

Выведем кусочек индекса в виде таблицы

In [193]:
inv_ind = inverted_index(norm_texts)

In [194]:
import pandas as pd

In [223]:
ind_df = pd.Series(inv_ind)
print(ind_df)

40                                             [7, 20, 98, 115, 147]
волшебник                                                       [52]
отступиться                                                    [113]
шутить             [0, 2, 4, 7, 8, 12, 15, 16, 17, 18, 27, 28, 37...
0                                                                 []
коляска                                                [11, 62, 151]
night                                                       [62, 89]
чрезвычайно                                                     [83]
устраиваться                                                [2, 128]
ночной                                        [7, 86, 110, 122, 126]
измельчить                                                      [57]
бергман                                                         [53]
должница                                                       [125]
перебить                                        [4, 61, 75, 95, 119]
чистить                           

С помощью обратного индекса произведите следующую аналитику:  

1) общая аналитика
- какое слово является самым частотным? === *ты* - 11164 раза во всех документах (код ниже) ===
- какое самым редким? === довольно много слов встречаются по одному разу. Например: *ещё*, *накидка*, *наклонить* и т.д. (тоже код ниже) ===
- какой набор слов есть во всех документах коллекции? === Мы знаем, что в коллекции 165 документов. Посмотрим, у каких слов в матрице inversed index длина индекса 165 ===

2) частота встречаемости имен главных героев в каждом сезоне      
- какой сезон был самым популярным у Чендлера? у Моники?   
- кто из главных героев статистически самый популярный? 


In [246]:
#ind_df.index = ind_df.str.len()
#ind_df = ind_df.sort_index(ascending=False).reset_index(drop=True)
#print(ind_df)

print(df.sum(axis=0).sort_values(ascending=False))
# df.sum(axis=0).sort_values(ascending=False).to_csv('frequencies_friends.csv', encoding='utf-8')

ты             11164
не              9437
что             8751
это             7501
быть            4762
он              4120
мы              3629
на              3234
так             3056
весь            3030
она             2995
да              2944
мочь            2798
нет             2705
как             2657
вы              2547
мой             2429
хотеть          2024
но              2012
ну              1662
бы              1595
знать           1592
они             1542
сказать         1534
же              1430
если            1324
думать          1285
вот             1284
просто          1235
этот            1189
               ...  
наклонить          1
наклеечка          1
надоумить          1
сёрфить            1
сёмкий             1
надувной           1
надыбай            1
наедаться          1
сям                1
нажарить           1
сюзя               1
наживка            1
сюжетный           1
сюдов              1
назло              1
назначаться        1
назойливый   

In [247]:
ind_df.to_csv("inversed_index.csv", encoding='utf-8')

Я просто сохранила обратный индекс и в notepad вбила в поиск [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164]. И получилось 35 совпадений. Значит ***35*** слов встречаются во всех документах.

In [261]:
new_df = pd.DataFrame()
new_df['words'] = list(inv_ind.keys())
new_df['inversed_index'] = ind_df
new_df[0:35]   # список слов, которые встречаются во всех документах

,words,inversed_index
0,40,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,волшебник,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
2,отступиться,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
3,шутить,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
4,0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
5,коляска,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
6,night,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
7,чрезвычайно,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
8,устраиваться,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
9,ночной,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."


2) частота встречаемости имен главных героев в каждом сезоне      
- какой сезон был самым популярным у Чендлера? у Моники?   
- кто из главных героев статистически самый популярный? 

In [267]:
print(normalize('Чендлер'), normalize('Моника'), normalize('Росс'),\
      normalize('Джои'), normalize('Рэйчел'), normalize('Фиби'))

чендлера моника росс джой рэйчел фиби


In [269]:
print(df['чендлера'].sum(axis=0))
print(df['моника'].sum(axis=0))
print(df['росс'].sum(axis=0), 'Ross')
print(df['джой'].sum(axis=0))
print(df['рэйчел'].sum(axis=0))
print(df['фиби'].sum(axis=0))

675
679
1012 Ross
69
236
572


Росс по всем сезонам статистически самый популярный из всех главных героев

## Функция ранжирования Okapi BM25

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Q$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:

$$ score(D, Q) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{(k_1+1)*f(q_i,D)}{f(q_i,D)+k_1(1-b+b\frac{|D|}{avgdl})} $$ 
где   
>$f(q_i,D)$ - частота слова $q_i$ в документе $D$ (TF)       
$|D|$ - длина документа (количество слов в нём)   
*avgdl* — средняя длина документа в коллекции    
$k_1$ и $b$ — свободные коэффициенты, обычно их выбирают как $k_1$=2.0 и $b$=0.75   
$$$$
$\text{IDF}(q_i)$ есть обратная документная частота (IDF) слова $q_i$: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в коллекции   
$n(q_i)$ — количество документов, содержащих $q_i$

In [36]:
### реализуйте эту функцию ранжирования 
from math import log

k1 = 2.0
b = 0.75

def score_BM25(qf, dl, avgdl, k1, b, N, n) -> float:
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    idf = log((N - ) / ())
    
    return 

### __Задача__:    
напишите функцию, которая сортирует поисковую выдачу для любого входящего запроса согласно метрике *Okapi BM25*.    
Выведите 10 первых результатов и их скор по запросу **рождественские каникулы**. 

In [38]:
def compute_sim() -> float:
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    return 


def get_search_result() -> list:
    """
    Compute sim score between search query and all documents in collection
    Collect as pair (doc_id, score)
    :param query: input text
    :return: list of lists with (doc_id, score)
    """
    return 